# Install Ultralytics library and remove "sample_data" folder

In [ ]:
!pip install ultralytics # install ultralytics to use YOLOv8
!rm -r sample_data # remove default sample_data google folder
import ultralytics
from ultralytics import YOLO
from google.colab import files # import google tools
import matplotlib.pyplot as plt
from PIL import Image

# DOWNLOAD MID-1K DATASET

In [ ]:
!git clone https://github.com/kennedyk1/MID-1K

# GENERATE DATASET CONFIG FILES (rgb.yaml and thermal.yaml)

In [ ]:
!echo "path: '/content/MID-1K/dataset'" > rgb.yaml
!echo "train: 'train/rgb/images'" >> rgb.yaml
!echo "val: 'test/rgb/images'" >> rgb.yaml
!echo "# class names" >> rgb.yaml
!echo "names:" >> rgb.yaml
!echo "  0: 'human'" >> rgb.yaml

!echo "path: '/content/MID-1K/dataset'" > thermal.yaml
!echo "train: 'train/thermal/images'" >> thermal.yaml
!echo "val: 'test/thermal/images'" >> thermal.yaml
!echo "# class names" >> thermal.yaml
!echo "names:" >> thermal.yaml
!echo "  0: 'human'" >> thermal.yaml

# CREATE YOLO RGB MODEL, TRAIN AND DOWNLOAD

In [ ]:
#!rm -r RML2024/RGB* #REMOVE OTHERS RGB FOLDERS
num_epochs = 50 # NUMBER OF EPOCHS TO TRAIN

#LOAD YOLOv8 SMALL MODEL FOR RGB MODALITY (nano, small, medium...)
model_RGB = YOLO('yolov8s.pt') #LOAD YOLO MODEL FOR TRAINNING Ex.: yolov8n.pt, yolov8s.pt, yolov8m.pt ...

#https://docs.ultralytics.com/usage/cfg/#train-settings
model_RGB.train(
    pretrained = False, #DEFINE IF USE PRETRAINED WEIGHTS
    data = 'rgb.yaml', #DATASET CONFIG FILE
    epochs = num_epochs, #NUMBER OF EPOCHS
    patience = num_epochs, #TO DISABLE EARLY STOP
    imgsz = 640, #TO REZISE IMAGES, DEFAULT 640
    augment=True,         # Enable data augmentation
    save = True, #TO SAVE CHECKPOINTS AND FINAL MODEL WEIGHTS
    project = 'RML2024', #NAME OF PROJECT
    name = 'RGB', #SUB-NAME OF PROJECT or MODALITY
    plots = True #TO SHOW PLOTS OF TRAINNING AND VALIDATION METRICS
)

#!zip -r RGB.zip /content/RML2024/RGB  #OPTIONAL TO ZIP
#files.download("RGB.zip")  #OPTIONAL TO DOWNLOAD TRAINING DATA AND TRAINED WEIGHTS

# CREATE YOLO THERMAL MODEL, TRAIN AND DOWNLOAD

In [ ]:
#!rm -r RML2024/THERMAL* # REMOVE OTHERS THERMAL FOLDERS
num_epochs = 50 # NUMBER OF EPOCHS TO TRAIN

#LOAD YOLOv8 SMALL MODEL FOR THERMAL MODALITY (nano, small, medium...)
model_THERMAL = YOLO('yolov8s.pt') #LOAD YOLO MODEL FOR TRAINNING Ex.: yolov8n.pt, yolov8s.pt, yolov8m.pt ...

#https://docs.ultralytics.com/usage/cfg/#train-settings
model_THERMAL.train(
    pretrained = False, #DEFINE IF USE PRETRAINED WEIGHTS
    data = 'thermal.yaml', #DATASET CONFIG FILE
    epochs = num_epochs, #NUMBER OF EPOCHS
    patience = num_epochs, #TO DISABLE EARLY STOP
    imgsz = 640, #TO REZISE IMAGES, DEFAULT 640
    augment=True,         # Enable data augmentation
    save = True, #TO SAVE CHECKPOINTS AND FINAL MODEL WEIGHTS
    project = 'RML2024', #NAME OF PROJECT
    name = 'THERMAL', #SUB-NAME OF PROJECT or MODALITY
    plots = True #TO SHOW PLOTS OF TRAINNING AND VALIDATION METRICS
)

#!zip -r THERMAL.zip /content/RML2024/THERMAL  #OPTIONAL TO ZIP
#files.download("THERMAL.zip")  #OPTIONAL TO DOWNLOAD TRAINING DATA AND TRAINED WEIGHTS

# RGB MODEL VALIDATION

In [ ]:
results_RGB = model_RGB.val()

## THERMAL MODEL VALIDATION

In [ ]:
results_thermal = model_THERMAL.val()

# TO PREDICT IMAGE. YOU CAN USE LOCAL FILES OR URL IMAGE

In [ ]:
#TUTORIAL: https://docs.ultralytics.com/modes/predict/
rgb_image = 'MID-1K/dataset/test/rgb/images/[09272023][112450]_852.png' # or https://mywebsite.com/image.png
thermal_image = 'MID-1K/dataset/test/thermal/images/[09272023][112450]_852.png' # or https://mywebsite.com/image.png

pred_RGB = model_RGB(rgb_image)
pred_thermal = model_THERMAL(thermal_image)

#CREATE FOLDER TO SAVE PREDICTIONS
try:
  import os
  os.makedirs('PREDICTIONS')
except:
  pass

id=0
for i in pred_RGB:
  cls = i.boxes.cls.tolist()
  conf = i.boxes.conf.tolist()
  boxes = i.boxes.xywhn.tolist()
  file = 'PREDICTIONS/RGB_result_'+str(id)+'.jpg'
  i.save(filename=file) #SAVE TO DISK IN FOLDER PREDICTIONS
  id = id+1

id=0
for i in pred_thermal:
  cls = i.boxes.cls.tolist()
  conf = i.boxes.conf.tolist()
  boxes = i.boxes.xywhn.tolist()
  file = 'PREDICTIONS/THERMAL_result_'+str(id)+'.jpg'
  i.save(filename=file) #SAVE TO DISK IN FOLDER PREDICTIONS
  id = id+1


#PLUS

In [ ]:
!pip install pillow

import os
from PIL import Image, ImageDraw, ImageFont # Import ImageFont
from ultralytics import YOLO

# Implement early fusion strategy
def early_fusion(pred_rgb, pred_thermal):
    combined_boxes = []
    combined_scores = []
    num_boxes = min(len(pred_rgb[0].boxes.xywh), len(pred_thermal[0].boxes.xywh)) # Find the minimum number of boxes
    for i in range(num_boxes):
        box_rgb = pred_rgb[0].boxes.xywh[i]
        box_thermal = pred_thermal[0].boxes.xywh[i]
        combined_box = (box_rgb + box_thermal) / 2  # Example fusion strategy: averaging boxes
        combined_boxes.append(combined_box)
        combined_score = (pred_rgb[0].boxes.conf[i] + pred_thermal[0].boxes.conf[i]) / 2
        combined_scores.append(combined_score)
    return combined_boxes, combined_scores

# Example usage of early fusion
combined_boxes, combined_scores = early_fusion(pred_RGB, pred_thermal)
print(f"Combined Boxes: {combined_boxes}")
print(f"Combined Scores: {combined_scores}")

# Create folder to save predictions if it doesn't exist
os.makedirs('PREDICTIONS', exist_ok=True)

# Function to draw bounding boxes on an image
def draw_boxes(image_path, boxes, scores, output_path, color, label):
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default() # Load a default font
    for box, score in zip(boxes, scores):
        # Calculate bounding box coordinates
        x_center, y_center, width, height = box
        left = (x_center - width / 2)
        top = (y_center - height / 2)
        right = (x_center + width / 2)
        bottom = (y_center + height / 2)
        draw.rectangle([left, top, right, bottom], outline=color, width=3)
        draw.text((left, top), f"{label} {score:.2f}", fill=color, font=font) # Draw the label and score
    image.save(output_path)
    return image

# Draw RGB and thermal bounding boxes on the RGB image, use rgb_image instead of rgb_image_path
draw_boxes(rgb_image, pred_RGB[0].boxes.xywh.tolist(), pred_RGB[0].boxes.conf.tolist(), 'PREDICTIONS/RGB_with_boxes.jpg', color='red', label='RGB')
draw_boxes(rgb_image, pred_thermal[0].boxes.xywh.tolist(), pred_thermal[0].boxes.conf.tolist(), 'PREDICTIONS/Thermal_with_boxes.jpg', color='blue', label='Thermal')

# Draw combined bounding boxes on the RGB image
draw_boxes(rgb_image, combined_boxes, combined_scores, 'PREDICTIONS/Combined_with_boxes.jpg', color='green', label='Combined')

# Download the annotated images
# Assuming 'files' is from google.colab, import it
from google.colab import files
files.download('PREDICTIONS/RGB_with_boxes.jpg')
files.download('PREDICTIONS/Thermal_with_boxes.jpg')
files.download('PREDICTIONS/Combined_with_boxes.jpg')

#DOWNLOAD

In [ ]:
!zip -r RGB.zip /content/RML2024/RGB  #OPTIONAL TO ZIP
files.download("RGB.zip")  #OPTIONAL TO DOWNLOAD TRAINING DATA AND TRAINED WEIGHTS

!zip -r THERMAL.zip /content/RML2024/THERMAL  #OPTIONAL TO ZIP
files.download("THERMAL.zip")  #OPTIONAL TO DOWNLOAD TRAINING DATA AND TRAINED WEIGHTS

# Zip the project folders (optional)
!zip -r ALL.zip /content/RML2024
files.download("ALL.zip")